# Why a solver wrapper should NOT try to enhance a solver capabilities

## Running example:

**Limitation of solver**: The solver supports `MOI.VectorAffineFunction{Float64}`-in-`MOI.Zeros` and `MOI.VectorOfVariables`-in-`S` but not `MOI.VectorAffineFunction{Float64}`-in-`S`.

**Enhancement in wrapper**: Support `f::MOI.VectorAffineFunction{Float64}`-in-`set::S` by creating slacks `slack` and add a constraint `slack`-in-`set` and (`slack - f`)-in-`MOI.Zeros(...)`.

## It is more work that it looks like

What if the user:
* ask for the number/list of variables (the `slack` variables should be hidden);
* ask for the number/list of constraints (the `slack`-in-`set` and (`slack - f`)-in-`MOI.Zeros(...)` should be hidden);
* modifies coefficients of `f`;
* modifies constants of `f`;
* modifies the set `S`;
* query primal/dual values;
* set start values for primal/dual;
* set a names...

It is a lot of code to maintain, it is better to write it once for all solvers than to duplicate the work.

## It is making it worse, not better

Suppose a user wants to write a solver-agnistic code and can either use `MOI.VectorOfVariables`-in-`S` or `MOI.VectorAffineFunction{Float64}`-in-`S`.
He wants to know which form is the most efficient, he does not want slack variables to be created in secret.
```julia
function f(optimizer::MOI.AbstractOptimizer)
    if MOI.supports(optimizer, MOI.VectorOfVariables, set)
        ...
    elseif MOI.supports(optimizer, MOI.VectorAffineFunction{Float64}, set)
        ...
    else
        error(...)
    end
end
```

More importantly, when a constraint is bridges, a Bellman-Ford-like algorithm is solved to find the way to
bridge it using the least amount of bridge possible, see the [JuMP-dev 2018 presentation](https://www.youtube.com/results?search_query=constraint+bridges) for more details.
If some bridges are done inside the wrapper, they cannot be taken into account and it misleads the Bellman-Ford algorithm.

## Conclusion

Solvers have different interfaces, which makes it impossible to write solver agnistic code.
MOI is a solver-agnistic interface that allows solvers to *speak the same language* to describe their capabilities and provide them.
MOI is *inclusive*, it is ok of a solver is different, it does not need to *pretend* in its MOI implementation.
An MOI implementation should not try to *fit* in a specific class.

Most importantly: A missing capability in the MOI implementation does **not** mean that it is not available when used with JuMP!